# ETL: FIFA World Cup 2022 Dataset

Ce notebook présente les étapes d'un processus ETL (Extract, Transform, Load) sur le jeu de données de la Coupe du Monde 2022.
L'objectif est d'analyser le dataset brut, de nettoyer et de sélectionner les colonnes pertinentes pour une analyse ultérieure.

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_columns', None)

## 1. Extraction et Chargement des Données
Nous téléchargeons la dernière version du dataset depuis Kaggle via `kagglehub`.

In [ ]:
# Download latest version
path = kagglehub.dataset_download("die9origephit/fifa-world-cup-2022-complete-dataset")

print("Path to dataset files:", path)

# Locate and read the CSV
csv_path = None
for root, dirs, files in os.walk(path):
    for file in files:
        if file == "Fifa_world_cup_matches.csv":
            csv_path = os.path.join(root, file)
            break

if csv_path:
    df = pd.read_csv(csv_path)
    print("Dataset loaded successfully.")
else:
    print("File not found.")

### 1.2 Analyse du Dataset (Exploration)
Dans cette étape, nous explorons la structure des données, vérifions les types et recherchons les valeurs manquantes.

In [ ]:
# Aperçu des premières lignes
df.head()

In [50]:
# Informations sur les colonnes et les types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 88 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   team1                                                  64 non-null     object
 1   team2                                                  64 non-null     object
 2   possession team1                                       64 non-null     object
 3   possession team2                                       64 non-null     object
 4   possession in contest                                  64 non-null     object
 5   number of goals team1                                  64 non-null     int64 
 6   number of goals team2                                  64 non-null     int64 
 7   date                                                   64 non-null     object
 8   hour                                                   64 non-

In [ ]:
# Vérification des valeurs nulles
nulls = df.isnull().sum()
print("Colonnes avec des valeurs nulles :")
print(nulls[nulls > 0])

In [49]:
# Statistiques descriptives sommaires
df.describe()

,number of goals team1,number of goals team2,total attempts team1,total attempts team2,conceded team1,conceded team2,goal inside the penalty area team1,goal inside the penalty area team2,goal outside the penalty area team1,goal outside the penalty area team2,assists team1,assists team2,on target attempts team1,on target attempts team2,off target attempts team1,off target attempts team2,attempts inside the penalty area team1,attempts inside the penalty area team2,attempts outside the penalty area team1,attempts outside the penalty area team2,left channel team1,left channel team2,left inside channel team1,left inside channel team2,central channel team1,central channel team2,right inside channel team1,right inside channel team2,right channel team1,right channel team2,total offers to receive team1,total offers to receive team2,inbehind offers to receive team1,inbehind offers to receive team2,inbetween offers to receive team1,inbetween offers to receive team2,infront offers to receive team1,infront offers to receive team2,receptions between midfield and defensive lines team1,receptions between midfield and defensive lines team2,attempted line breaks team1,attempted line breaks team2,completed line breaksteam1,completed line breaks team2,attempted defensive line breaks team1,attempted defensive line breaks team2,completed defensive line breaksteam1,completed defensive line breaks team2,yellow cards team1,yellow cards team2,red cards team1,red cards team2,fouls against team1,fouls against team2,offsides team1,offsides team2,passes team1,passes team2,passes completed team1,passes completed team2,crosses team1,crosses team2,crosses completed team1,crosses completed team2,switches of play completed team1,switches of play completed team2,corners team1,corners team2,free kicks team1,free kicks team2,penalties scored team1,penalties scored team2,goal preventions team1,goal preventions team2,own goals team1,own goals team2,forced turnovers team1,forced turnovers team2,defensive pressures applied team1,defensive pressures applied team2
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.00000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.0,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.0000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000
mean,1.578125,1.109375,11.140625,11.281250,1.109375,1.578125,1.468750,0.984375,0.093750,0.109375,1.171875,0.734375,4.203125,3.750000,4.703125,5.031250,6.93750,6.953125,4.203125,4.328125,13.625000,13.500000,4.921875,4.421875,4.656250,5.156250,4.265625,5.031250,11.656250,12.796875,592.37500,550.218750,126.375000,119.625000,231.500000,212.859375,234.500000,217.734375,11.406250,10.500000,173.46875,166.593750,114.250000,106.484375,18.484375,18.265625,10.156250,9.718750,1.781250,1.750000,0.062500,0.0,12.640625,12.359375,1.968750,1.968750,509.515625,492.109375,437.0000,419.890625,18.093750,18.531250,4.593750,4.078125,6.453125,6.156250,4.484375,4.453125,14.093750,14.390625,0.140625,0.125000,11.593750,11.359375,0.015625,0.015625,71.968750,70.125000,289.750000,293.265625
std,1.551289,1.055856,4.972519,5.807682,1.055856,1.551289,1.563155,0.999876,0.293785,0.314576,1.363407,0.895176,2.527184,2.713868,2.394966,2.911219,3.77912,4.459446,2.470009,2.766321,6.550173,7.287737,2.534240,3.201213,2.852004,3.296071,2.685896,3.141978,5.812463,6.544547,170.21084,169.487694,33.776812,36.660822,70.466698,59.487191,85.887893,101.472843,6.920682,5.614607,32

## 2. Transformation des Données (ETL)
Cette étape consiste à nettoyer les données brutes, à sélectionner les attributs pertinents et à enrichir le dataset si nécessaire.

### 2.1 Sélection des colonnes
Nous conservons uniquement les colonnes pertinentes pour notre analyse (équipes, scores, statistiques clés).

In [ ]:
columns_to_keep = [
    'team1', 'team2', 'date', 'hour', 'category',
    'number of goals team1', 'number of goals team2',
    'possession team1', 'possession team2', 'possession in contest',
    'total attempts team1', 'total attempts team2',
    'on target attempts team1', 'on target attempts team2',
    'yellow cards team1', 'yellow cards team2',
    'red cards team1', 'red cards team2',
    'penalties scored team1', 'penalties scored team2'
]

df_selected = df[columns_to_keep].copy()
df_selected.head()

### Justification des choix

Nous choisissons de garder les colonnes suivantes :

1.  **Informations Générales** :
    *   `team1`, `team2` : Les équipes qui s'affrontent.
    *   `date`, `hour`, `category` : Contexte temporel et phase du tournoi (e.g., Groupe, Finale).

2.  **Résultat du Match** :
    *   `number of goals team1`, `number of goals team2` : Indispensable pour connaître le vainqueur.

3.  **Statistiques de Jeu (Performance)** :
    *   `possession team1`, `possession team2` : Indicateur clé de domination.
    *   `total attempts team1`, `total attempts team2` : Volume offensif.
    *   `on target attempts team1`, `on target attempts team2` : Précision et dangerosité réelle.

4.  **Discipline** :
    *   `yellow cards team1`, `yellow cards team2`
    *   `red cards team1`, `red cards team2` : Impact sur le jeu et fair-play.
    *   `penalties scored...` : Pour identifier les matchs aux tirs au but.

Ces colonnes nous permettront de répondre aux questions principales : Qui a gagné ? Qui a dominé ? Le match était-il agressif ?

### 2.2 Nettoyage et Conversion de Types
Les colonnes de possession contiennent le caractère `%` et sont de type objet (string). Nous devons les convertir en valeurs numériques.
De plus, la colonne `date` doit être convertie au format `datetime` pour permettre des analyses temporelles.

In [ ]:
# Conversion des pourcentages (suppression du % et conversion en float)
def clean_percentage(x):
    if isinstance(x, str):
        return float(x.replace('%', ''))
    return x

cols_percent = ['possession team1', 'possession team2', 'possession in contest']
for col in cols_percent:
    df_selected[col] = df_selected[col].apply(clean_percentage)

# Conversion de la date
df_selected['date'] = pd.to_datetime(df_selected['date'], format='%Y-%m-%d')

df_selected.info()

### 2.3 Feature Engineering (Enrichissement)

Création de nouvelles colonnes utiles pour l'analyse :
*   `total_goals` : Nombre total de buts dans le match.
*   `goal_difference` : Écart de buts (valeur absolue).

In [ ]:
df_selected['total_goals'] = df_selected['number of goals team1'] + df_selected['number of goals team2']
df_selected['goal_difference'] = abs(df_selected['number of goals team1'] - df_selected['number of goals team2'])

print("Aperçu des données enrichies :")
df_selected.head()

Les données sont maintenant propres et enrichies, prêtes pour l'analyse.

In [47]:

equipes = pd.concat([df_selected['team1'], df_selected['team2']]).unique().tolist()
print(equipes, len(equipes))

['QATAR', 'ENGLAND', 'SENEGAL', 'UNITED STATES', 'ARGENTINA', 'DENMARK', 'MEXICO', 'FRANCE', 'MOROCCO', 'GERMANY', 'SPAIN', 'BELGIUM', 'SWITZERLAND', 'URUGUAY', 'PORTUGAL', 'BRAZIL', 'WALES', 'NETHERLANDS', 'TUNISIA', 'POLAND', 'JAPAN', 'CROATIA', 'CAMEROON', 'KOREA REPUBLIC', 'ECUADOR', 'IRAN', 'AUSTRALIA', 'SAUDI ARABIA', 'CANADA', 'COSTA RICA', 'GHANA', 'SERBIA'] 32


### 2.4 Adaptation au Schéma SQL (Matches)

Nous transformons le DataFrame pour correspondre exactement à la structure de la table SQL `matches` cible. 
Cela inclut la génération d'IDs pour les équipes, le renommage des colonnes et la création des indicateurs manquants.

In [51]:
# Génération d'IDs pour les équipes (mapping simple basé sur l'ordre alphabétique)
# Dans un cas réel, ces IDs viendraient d'une table 'teams' déjà peuplée
unique_teams = sorted(pd.concat([df_selected['team1'], df_selected['team2']]).unique())
# team_mapping = {team: i+1 for i, team in enumerate(unique_teams)}

df_matches = df_selected.copy()

# Création des Foreign Keys
df_matches['home_team_id'] = df_matches['team1']
df_matches['away_team_id'] = df_matches['team2']

# Renommage des colonnes
df_matches = df_matches.rename(columns={
    'number of goals team1': 'home_result',
    'number of goals team2': 'away_result',
    'category': 'round'
})

# Calcul de la colonne 'result'
conditions = [
    df_matches['home_result'] > df_matches['away_result'],
    df_matches['home_result'] < df_matches['away_result']
]
choices = ['home_team', 'away_team']
df_matches['result'] = np.select(conditions, choices, default='draw')

# Gestion des booléens
df_matches['penalties'] = (df_matches['penalties scored team1'] + df_matches['penalties scored team2']) > 0
df_matches['extra_time'] = df_matches['penalties'] # Assumption simplifiée
df_matches['replay'] = False

# Colonnes statiques ou manquantes
df_matches['edition'] = 2022
df_matches['city'] = None # Information non disponible dans ce dataset
df_matches['id_stadium'] = None # Information non disponible

# Génération d'un ID de match séquentiel
df_matches['id_match'] = range(1, len(df_matches) + 1)

# Sélection et ordonnancement final des colonnes selon le schéma SQL
final_cols = [
    'id_match', 
    'home_team_id', 'away_team_id', 
    'home_result', 'away_result', 'result', 
    'extra_time', 'penalties', 'replay', 
    'date', 'round', 'city', 'id_stadium', 'edition'
]

df_matches_final = df_matches[final_cols].copy()

print("Aperçu du DataFrame final prêt pour l'insertion SQL :")
df_matches_final.head()

Aperçu du DataFrame final prêt pour l'insertion SQL :


,id_match,home_team_id,away_team_id,home_result,away_result,result,extra_time,penalties,replay,date,round,city,id_stadium,edition
0,1,QATAR,ECUADOR,0,2,away_team,True,True,False,2022-11-20,Group A,None,None,2022
1,2,ENGLAND,IRAN,6,2,home_team,True,True,False,2022-11-21,Group B,None,None,2022
2,3,SENEGAL,NETHERLANDS,0,2,away_team,False,False,False,2022-11-21,Group A,None,None,2022
3,4,UNITED STATES,WALES,1,1,draw,True,True,False,2022-11-21,Group B,None,None,2022
4,5,ARGENTINA,SAUDI ARABIA,1,2,away_team,True,True,False,2022-11-22,Group C,None,None,2022


In [ ]:
import requests
import json
import os  # Nécessaire pour gérer les chemins de dossiers

# 1. URL pour les données en Anglais
url = "https://api.fifa.com/api/v3/calendar/matches?language=en&count=500&idSeason=255711"

# 2. Configuration (Headers)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Origin': 'https://www.fifa.com',
    'Referer': 'https://www.fifa.com/'
}

print("⏳ Téléchargement des données en Anglais...")

try:
    # 3. Récupération
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    # --- PARTIE MODIFIÉE : GESTION DU DOSSIER DATA ---
    
    # On définit le nom du fichier
    nom_fichier = "matches_wc2022_en.json"
    
    # On définit le chemin : ".." signifie "remonter d'un dossier", puis aller dans "data"
    # os.path.join permet que ça marche sur Windows (anti-slash) et Mac/Linux (slash)
    chemin_dossier = os.path.join("..", "data/raw")
    chemin_complet = os.path.join(chemin_dossier, nom_fichier)

    # Sécurité : On crée le dossier s'il n'existe pas (même si vous l'avez déjà)
    os.makedirs(chemin_dossier, exist_ok=True)
    
    # 4. Sauvegarde dans le dossier spécifique
    with open(chemin_complet, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
    # On affiche le chemin absolu pour confirmer l'emplacement
    print(f"✅ Succès ! Fichier enregistré ici :")
    print(os.path.abspath(chemin_complet))

except Exception as e:
    print(f"❌ Erreur : {e}")

⏳ Téléchargement des données en Anglais...
✅ Succès ! Fichier enregistré ici :
C:\Users\mabed\Documents\Brief-2-ETL-donnees-footballistiques-Short-Kings\data\matches_wc2022_en.json


In [ ]:
import json
import os
import pandas as pd
import numpy as np

# 1. Chargement du JSON
json_path = os.path.join('..', 'data', 'matches_wc2022_en.json')
with open(json_path, 'r', encoding='utf-8') as f:
    wc_data = json.load(f)

# 2. Fonction de nettoyage des rounds
def clean_round_name(r_raw):
    r = str(r_raw).upper().strip()
    # On gère les majuscules pour éviter le bug de la 3ème place
    if 'PRELIMINARY' in r or 'FIRST' in r: return 'Preliminary'
    if 'GROUP' in r: return 'Group Stage'
    if '1/8' in r or 'ROUND OF 16' in r: return 'Round of 16'
    if '1/4' in r or 'QUARTER' in r: return 'Quarter-finals'
    if '1/2' in r or 'SEMI' in r: return 'Semi-final'
    if '3RD' in r or 'THIRD' in r or 'PLAY-OFF' in r: return 'Third Place'
    if 'FINAL' in r: return 'Final'
    return r_raw

# 3. Extraction
extracted_data = []

for match in wc_data['Results']:
    home = match['Home']['TeamName'][0]['Description'].upper()
    away = match['Away']['TeamName'][0]['Description'].upper()
    
    stage = match['StageName'][0]['Description']
    group = match['GroupName'][0]['Description'] if match['GroupName'] else None
    round_raw = group if (stage == "First stage" and group) else stage
    
    # --- MODIFICATION ICI ---
    # A. Récupération des scores du JEU (90 min + Prolongations)
    # L'API FIFA stocke généralement le score hors tirs au but dans HomeTeamScore
    score_board_home = match.get('HomeTeamScore', 0)
    score_board_away = match.get('AwayTeamScore', 0)
    
    # B. Récupération des pénaltys
    pen_home = match.get('HomeTeamPenaltyScore', 0)
    pen_away = match.get('AwayTeamPenaltyScore', 0)
    is_penalties = (pen_home > 0 or pen_away > 0)
    
    # C. Calcul du score TOTAL (pour déterminer le vainqueur uniquement)
    total_calc_home = score_board_home + pen_home
    total_calc_away = score_board_away + pen_away

    extracted_data.append({
        'home_team_id': home,
        'away_team_id': away,
        'round': clean_round_name(round_raw),
        'city': match['Stadium']['CityName'][0]['Description'],
        'id_stadium': match['Stadium']['Name'][0]['Description'],
        'extra_time': is_penalties,
        'penalties': is_penalties,
        
        # Ce que l'utilisateur verra (Score du match)
        'home_result': score_board_home, 
        'away_result': score_board_away,
        
        # Colonnes temporaires pour le calcul du gagnant (incluant TAB)
        'temp_total_home': total_calc_home,
        'temp_total_away': total_calc_away
    })

# 4. DataFrame JSON
df_json = pd.DataFrame(extracted_data)

# 5. Fusion
cols_to_drop = ['round', 'city', 'id_stadium', 'extra_time', 'penalties', 'home_result', 'away_result']
df_matches_final = df_matches_final.drop(columns=[c for c in cols_to_drop if c in df_matches_final.columns], errors='ignore')

# Important : Assurez-vous que les IDs sont bien en majuscules dans le DF principal aussi
df_matches_final['home_team_id'] = df_matches_final['home_team_id'].str.upper().str.strip()
df_matches_final['away_team_id'] = df_matches_final['away_team_id'].str.upper().str.strip()

df_matches_final = df_matches_final.merge(df_json, on=['home_team_id', 'away_team_id'], how='left')

# 6. Calcul du RESULT (Vainqueur)
# On utilise les colonnes temporaires (qui incluent les pénaltys)
conditions = [
    (df_matches_final['temp_total_home'] > df_matches_final['temp_total_away']),
    (df_matches_final['temp_total_home'] < df_matches_final['temp_total_away'])
]
choices = ['home_team', 'away_team']
df_matches_final['result'] = np.select(conditions, choices, default='draw')

# 7. NETTOYAGE FINAL
# On supprime les colonnes de calcul temporaires, on ne garde que les scores "officiels" du match
df_matches_final = df_matches_final.drop(columns=['temp_total_home', 'temp_total_away'])

# Affichage
print("Données enrichies : Scores (hors TAB) affichés, mais Vainqueur (avec TAB) calculé correctemen t:")
# On affiche un match spécifique qui a fini aux tirs au but (ex: Finale France-Argentine) pour vérifier
display(df_matches_final[df_matches_final['penalties'] == True].head())

Données enrichies et optimisées :


,id_match,home_team_id,away_team_id,result,replay,date,edition,round,city,id_stadium,extra_time,penalties,home_result,away_result
0,1,QATAR,ECUADOR,away_team,False,2022-11-20,2022,Group Stage,Al Khor,Al Bayt Stadium,False,False,0,2
1,2,ENGLAND,IRAN,home_team,False,2022-11-21,2022,Group Stage,Ar-Rayyan,Khalifa International Stadium,False,False,6,2
2,3,SENEGAL,NETHERLANDS,away_team,False,2022-11-21,2022,Group Stage,Doha,Al Thumama Stadium,False,False,0,2
3,4,UNITED STATES,WALES,draw,False,2022-11-21,2022,Group Stage,Ar-Rayyan,Ahmad Bin Ali Stadium,False,False,1,1
4,5,ARGENTINA,SAUDI ARABIA,away_team,False,2022-11-22,2022,Group Stage,Al Daayen,Lusail Stadium,False,False,1,2
5,6,DENMARK,TUNISIA,draw,False,2022-11-22,2022,Group Stage,Doha,Education City Stadium,False,False,0,0
6,7,MEXICO,POLAND,draw,False,2022-11-22,2022,Group Stage,Doha,Stadium 974,False,False,0,0
7,8,FRANCE,AUSTRALIA,home_team,False,2022-11-22,2022,Group Stage,Al Wakrah,Al Janoub Stadium,False,False,4,1
8,9,MOROCCO,CROATIA,draw,False,2022-11-23,2022,Group Stage,Al Khor,Al Bayt Stadium,False,False,0,0
9,10,GERMANY,JAPAN,away_team,False,2022-11-23,2022,Group Stage,Ar-Rayyan,Khalifa International Stadium,False,False,1,2


In [79]:
df_matches_final.to_csv('../data/processed/df_matches_final.csv', index=False)
